# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [1]:
#!pip install chromadb

  Using cached opentelemetry_api-1.27.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.27.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached opentelemetry_instrumentation_fastapi-0.48b0-py3-none-any.whl.metadata (2.1 kB)
  Using cached opentelemetry_sdk-1.27.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached PyPika-0.48.9.tar.gz (67 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached importlib_resources-6.4.5-py3-none-any.whl.metadata (4.0 kB)
  Using cached bcrypt-4.2.0-cp39-abi3-win_amd64.whl.metadata (9.9 kB)
  Using cached typer-0.12.5-py3-none-any.whl.metadata (15 kB)
  Using cac

In [11]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from bs4 import BeautifulSoup

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
#HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [4]:
llm = OpenAI(temperature=0)

In [5]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "sonnets.txt")

In [6]:
loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="sonnets")

In [7]:
sonnets = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [8]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [9]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [12]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

Created a chunk of size 2130, which is longer than the specified 1000
Created a chunk of size 3187, which is longer than the specified 1000
Created a chunk of size 1256, which is longer than the specified 1000
Created a chunk of size 1051, which is longer than the specified 1000


## Create the Agent

In [13]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [14]:
tools = [
    Tool(
        name="Sonnets QA System",
        func=sonnets.run,
        description="useful for when you need to answer questions about Shakespeare and his sonnets. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [15]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

C:\Users\katha\AppData\Local\Temp\ipykernel_14540\1834837320.py:3: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent = initialize_agent(


In [16]:
agent.invoke(
    "What did Shakespeare say about love?"
)



> Entering new AgentExecutor chain...
 Shakespeare is known for his sonnets about love, so the Sonnets QA System would likely be the best tool to use.
Action: Sonnets QA System
Action Input: "What did Shakespeare say about love?"
Observation:  Shakespeare believed that true love is constant and unchanging, and that it can withstand the test of time and any obstacles that may come its way. He also believed that love should not be based on superficial things like physical appearance, but rather on a deep and unwavering connection between two people.
Thought: This answer aligns with Shakespeare's famous quote, "Love looks not with the eyes, but with the mind."
Final Answer: Shakespeare believed that true love is constant and unchanging, and that it should be based on a deep and unwavering connection between two people rather than superficial things like physical appearance.

> Finished chain.


{'input': 'What did Shakespeare say about love?',
 'output': 'Shakespeare believed that true love is constant and unchanging, and that it should be based on a deep and unwavering connection between two people rather than superficial things like physical appearance.'}

In [17]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 It's important to understand the differences between tools before deciding which one to use.
Action: Ruff QA System
Action Input: "What are the differences between ruff and flake8?"
Observation:  Ruff implements more rules than Flake8 and does not support custom lint rules. It also has some minor differences in its rule set and does not include all of the 'opinionated' rules from flake8-bugbear. Additionally, Ruff is capable of automatically fixing its own lint violations, while Flake8 supports plugins and allows for custom and third-party rules.
Thought: It seems like Ruff may be a more comprehensive and customizable linter.
Action: Ruff QA System
Action Input: "What are the benefits of using Ruff?"
Observation: 
Some potential benefits of using Ruff include:

1. Compatibility with Black: Ruff is designed to be used alongside Black, making it easy to use both tools together for consistent code formatting.

2. Large rule set: Ruff implements ove

{'input': 'Why use ruff over flake8?',
 'output': "Ruff may be a more comprehensive and customizable linter, but it does have limitations such as not supporting custom lint rules. It's important to weigh these factors when deciding whether to use Ruff or another tool like Flake8."}

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [18]:
tools = [
    Tool(
        name="Sonnets QA System",
        func=sonnets.run,
        description="useful for when you need to answer questions about Shakespeare and his sonnets. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [19]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [20]:
agent.invoke(
    "What did Shakespeare say about love?"
)



> Entering new AgentExecutor chain...
 Shakespeare is known for his sonnets about love, so the Sonnets QA System would likely be the best tool to use.
Action: Sonnets QA System
Action Input: "What did Shakespeare say about love?"
Observation:  Shakespeare believed that true love is constant and unchanging, and that it can withstand the test of time and any obstacles that may come its way. He also believed that love should not be based on superficial things like physical appearance, but rather on a deep and unwavering connection between two people.


> Finished chain.


{'input': 'What did Shakespeare say about love?',
 'output': ' Shakespeare believed that true love is constant and unchanging, and that it can withstand the test of time and any obstacles that may come its way. He also believed that love should not be based on superficial things like physical appearance, but rather on a deep and unwavering connection between two people.'}

In [21]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 It's important to understand the differences between different tools before deciding which one to use.
Action: Ruff QA System
Action Input: "What are the differences between ruff and flake8?"
Observation:  Ruff implements more rules than Flake8 and does not support custom lint rules. It also has some minor differences in its rule set and does not include all of the 'opinionated' rules from flake8-bugbear. Additionally, Ruff is capable of automatically fixing its own lint violations, while Flake8 supports plugins and allows for custom and third-party rules.


> Finished chain.


{'input': 'Why use ruff over flake8?',
 'output': " Ruff implements more rules than Flake8 and does not support custom lint rules. It also has some minor differences in its rule set and does not include all of the 'opinionated' rules from flake8-bugbear. Additionally, Ruff is capable of automatically fixing its own lint violations, while Flake8 supports plugins and allows for custom and third-party rules."}

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [22]:
tools = [
    Tool(
        name="Sonnets QA System",
        func=sonnets.run,
        description="useful for when you need to answer questions about Shakespeare and his sonnets. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [23]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [24]:
agent.invoke(
    "What tool does ruff use to run over Jupyter Notebooks? Did Shakespeare mention that tool in his sonnets?"
)



> Entering new AgentExecutor chain...
 I should use the Ruff QA System to answer the first question and the Sonnets QA System to answer the second question.
Action: Ruff QA System
Action Input: What tool does ruff use to run over Jupyter Notebooks?
Observation:  Ruff does not currently support Jupyter Notebooks.
Thought: I should try asking about Jupyter Notebooks in the Sonnets QA System.
Action: Sonnets QA System
Action Input: Did Shakespeare mention Jupyter Notebooks in his sonnets?
Observation:  No, Shakespeare did not mention Jupyter Notebooks in his sonnets.
Thought: I should try asking about tools in general in the Sonnets QA System.
Action: Sonnets QA System
Action Input: Did Shakespeare mention any tools in his sonnets?
Observation:  No, Shakespeare does not mention any tools in his sonnets.
Thought: I now know the final answer.
Final Answer: No, Shakespeare did not mention any tools in his sonnets.

> Finished chain.


{'input': 'What tool does ruff use to run over Jupyter Notebooks? Did Shakespeare mention that tool in his sonnets?',
 'output': 'No, Shakespeare did not mention any tools in his sonnets.'}